# MODULES

In [ ]:
import panel as pn
import panel.widgets as pnw
pn.extension('plotly') #para que nos despliegue en nuestro notebook, activamos la extension de panel

import pandas as pd
import numpy as np

import plotly.graph_objects as go

import math